# Data Cleaning

This file stores all the codings related to data cleaning.

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Load raw data
accident = pd.read_csv("./Raw_data/accident.csv", encoding='cp1252')
accident.head(5)

# person = pd.read_csv("./Raw_data/person.csv", encoding='cp1252', errors='replace')
try:
    person = pd.read_csv("./Raw_data/person.csv", encoding='utf-8')
except UnicodeDecodeError:
    person = pd.read_csv("./Raw_data/person.csv", encoding='cp1252', errors='replace')
person.head(5)

,STATE,STATENAME,ST_CASE,VEH_NO,PER_NO,VE_FORMS,COUNTY,MONTH,MONTHNAME,DAY,...,DEVTYPE,DEVTYPENAME,DEVMOTOR,DEVMOTORNAME,LOCATION,LOCATIONNAME,WORK_INJ,WORK_INJNAME,HISPANIC,HISPANICNAME
0,1,Alabama,10001,1,1,1,13,1,January,4,...,NaN,NaN,NaN,NaN,0,Occupant of a Motor Vehicle,0,No,7,Non-Hispanic
1,1,Alabama,10002,1,1,1,97,3,March,20,...,NaN,NaN,NaN,NaN,0,Occupant of a Motor Vehicle,0,No,7,Non-Hispanic
2,1,Alabama,10003,1,1,1,89,3,March,18,...,NaN,NaN,NaN,NaN,0,Occupant of a Motor Vehicle,0,No,7,Non-Hispanic
3,1,Alabama,10004,1,1,1,53,3,March,17,...,NaN,NaN,NaN,NaN,0,Occupant of a Motor Vehicle,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable)
4,1,Alabama,10004,1,2,1,53,3,March,17,...,NaN,NaN,NaN,NaN,0,Occupant of a Motor Vehicle,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable)


In [3]:
interested_features = ['ST_CASE', 'STATE', 'PEDS', 'VE_TOTAL', 'PERSONS', 'COUNTY', 'DAY_WEEK', 'HOUR', 'ROUTE', 'FUNC_SYS', 'LATITUDE',
                       'LONGITUD', 'TYP_INT', 'REL_ROAD', 'LGT_COND', 'WEATHER', 'FATALS']

# Filter
accident = accident[interested_features]

# Remove unknown
accident = accident[(accident['COUNTY'] != 999) & (accident['HOUR'] != 99) & (accident['ROUTE'] != 99) & (accident['ROUTE'] != 95) & (accident['FUNC_SYS'] != 99)
                    & (accident['FUNC_SYS'] != 98) & (accident['LATITUDE'] != 77.7777) & (accident['LATITUDE'] != 99.9999) & (accident['LATITUDE'] != 88.8888)
                    & (accident['LONGITUD'] != 777.7777) & (accident['LONGITUD'] != 999.9999) & (accident['LONGITUD'] != 888.8888) & (accident['TYP_INT'] != 99)
                    & (accident['TYP_INT'] != 98) & (accident['REL_ROAD'] != 99) & (accident['REL_ROAD'] != 98) & (accident['LGT_COND'] != 9) & 
                    (accident['LGT_COND'] != 8) & (accident['LGT_COND'] != 7) & (accident['WEATHER'] != 99) & (accident['WEATHER'] != 98) &
                    (accident['WEATHER'] != 8)]

person = person[(person['INJ_SEV'] != 6) & (person['INJ_SEV'] != 0) & (person['INJ_SEV'] != 9)]
                    
# Group by ST_CASE and count INJ_SEV
injury_counts = person.groupby('ST_CASE').size().reset_index(name='NUM_INJURED')

# Merge with accident based on cases
accident = accident.merge(injury_counts, on='ST_CASE', how='left')
accident['NUM_INJURED'] = accident['NUM_INJURED'].fillna(0).astype(int)

accident.to_csv("./Processed_data/filtered_accident.csv", index=False)